# **Lab One: Vizualization and Data Preprocessing**

*Contributors:* Balaji Avvaru, Joshua Eysenbach, Vijay Kaniti, Daniel Turner

## **Business Understanding**

This analysis uses a dataset categorizing patients with a cardiovascular disease diagnosis.  It contains a collection of 11 attributes that were gathered with the intention of trying to identify potential characteristics of individuals that correlate with heart disease. The primary goal of this analysis is to explore the data through statistical summaries and visualization to elucidate any trends that will be useful for building a prediction model for classifying a patient as having a cardiovascular disease diagnosis or not.

This dataset was procured from Kaggle (https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) but it is unclear from where the original data originates. However, the features are described well enough to understand what each of them represents. In a practical "real-world" setting, verifying the source of this data and scrutinizing the methods of its collection would be a vital part of the analysis, but for the purposes of our academic interests in data visualization and eventual prediction modelling, this level of validity is inconsequential as long as the data has realistic characteristics and application.

The goal of any prediction algorithm using this data is to determine if any of these attributes or combination of them can predict a cardiovascular disease diagnosis. These predicition models could provide valuable insight into what conditions or behaviors might be correlated with heart disease and could be used in aiding diagnosis or helping to mitigate the disease through understanding its possible causes. Many of the attributes collected are based on long standing suppositions of conditions or behaviors associated with heart disease like high cholesterol, smoking, or alcohol consumption. This analysis and subsequent prediction modelling can hopefully identify if any of these are more associated with (or even perhaps falsely attributed to) a diagnosis.

The effectiveness of a prediction model for classifying patients could be measured in several ways depending on the implementation. For example, the intent of the model could be to identify individuals likely to be diagnosed with cardiovascular disease so they can be given an objective reason to make behavioral changes to reduce their chances of a future diagnosis. In this case, maximizing sensitivity at the expense of accuracy and specificity might be the best option because there would be few downsides to making false positive classifications. On the other hand, if something like high cholesterol was found to be a highly significant predictor and there was a decision to be made about prescribing a drug that could have side effects for the patient, accuracy and a more balanced sensitivity and specificity might be more important as false positives become more of a concern. For reasons like these, a few different prediction models might be warranted for different specific uses. 





## **Data Understanding**

### Data Meaning and Type - Feature Descriptions

The features included are described on the Kaggle page for this data as being separable into three categories:
* *Objective*: Factual initial information about the patient;
* *Examination*: Information resulting from medical examination;
* *Subjective*: Information given by the patient.

Distinctions between the different attributes are paramount to interpreting and qualifying results of analysis and modelling of this data as they can represent varying degrees of validity and potential biases, so it is important that we keep this in mind as we explore the data and eventually make any recommendations. For example, we would likely assume blood pressure measurements will be reasonably accurate as they were taken by a trained health professional, but should be wary of a patient's proclivity to be honest when asked whether they are a regular smoker or drinker.

The attributes included in the dataset are outlined below per their descriptions on the Kaggle page.

|Attribute Name | Category | Description |
|---------------|----------|-------------|
|age | Objective | Age of the Patient |
|height | Objective | Height of the Patient (cm) |
|weight | Objective | Weight of the Patient (kg) |
|gender | Objective | Gender of the Patient (1:M, 2:F)|
|ap_hi | Examination | Systolic blood pressure |
|ap_lo | Examination | Diastolic blood pressure |
|cholesterol | Examination | Cholesterol level -  1: normal, 2: above normal, 3: well above normal |
|gluc | Examination | Glucose level - 1: normal, 2: above normal, 3: well above normal |
|smoke | Subjective | Patient does (1) or does not (0) describe themselves as a smoker |
|alco | Subjective | Patient does (1) or does not (0) regularly drink alcohol |
|active | Subjective | Patient does (1) or does not (0) regularly exercise |
|cardio | Target Variable | Diagnosis of presence (1) or absence (0) of cardiovascular disease |

### Data Quality Verification

#### Initial Import

The dataset acquired from Kaggle is stored for our use on Github. The code for importing the data is combined with the inital loading of *Pandas* and *Numpy* packages below. 

In [18]:
import pandas as pd
import numpy as np

In [23]:
cd = pd.read_csv("https://raw.githubusercontent.com/jteysen/MSDS-7331-Machine-Learning-I/master/Data/cardio_train.csv")

We can verify that the import was successful and get a preview of our data with the *.head()* command. The .info() command is used to view the current data type of each attribute.

In [24]:
cd.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


The *.info()* command is used to view the current data type of each attribute.

In [25]:
cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


Note that there are no missing values in the data as there are 70,000 non-null values for all attributes. This also indicates (along with what we saw from the *.head()* command) that since all attributes are integers (or floats) that the categorical information is already in a numeric coded form.